# Snorkel

## Wprowadzenie

Celem laboratorium jest zapoznanie studentów z narzędziem [Snorkel](www.snorkel.org) i możliwościami programistycznego generowania etykiet korzystając z paradygmatu uczenia słabo-nadzorowanego.

W celu wykorzystania uczenia słabo-nadzorowanego do generowania etykiet konieczne jest stworzenie trzech zbiorów danych:

* **train set**: zbiór uczący, nie posiadający żadnych etykiet
* **validation set**: zbiór walidacyjny, wykorzystywany do optymalizacji hiperparametrów, posiada etykiety
* **test set**: zbiór testowy, wykorzystywany jedynie do ostatecznej ewaluacji modelu, posiada etykiety

## Funkcje etykietujące

Pierwszym krokiem będzie załadowanie zbioru danych oraz dokonanie podziału na zbiór uczący i zbiór testowy. Ponieważ w naszym zbiorze wszystkie SMS-y posiadają etykietę, zasymulujemy problem uczenia słabo-nadzorowanego przez losowe usunięcie 80% etykiet. Dodatkowo, Snorkel wymaga etykiet numerycznych, więc musimy dokonać przekodowania wartości.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1


df = pd.read_csv('/content/drive/My Drive/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])
df.head()

In [ ]:
df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

In [ ]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

df_test.head()

### Proste wyszukiwanie na podstawie słowa kluczowego

Jako pierwszy przykład wykorzystamy wyszukanie słów "check" i "free" w treściach SMS-ów

In [ ]:
!pip install snorkel

In [ ]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

Kolejnym krokiem jest zaaplikowanie funkcji etykietujących do zbioru uczącego.

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

Wynikiem zaaplikowania zbioru funkcji etykietujących do zbioru uczącego jest macierz o rozmiarze $m \times n$, gdzie $m$ to liczba przykładów, a $n$ to liczba funkcji etykietujących. Macierz zawiera wynik zastosowania każdej funkcji do każdego przykładu.

In [ ]:
L_train

Najprostszym sposobem analizy jest wyznaczenie pokrycia funkcji etykietujących (czyli procent przypadków, dla których funkcja zwróciła wynik inny niż `ABSTAIN`.

In [ ]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Pokrycie dla funkcji check(): {coverage_check * 100:.1f}%")
print(f"Pokrycie dla funkcji free(): {coverage_free * 100:.1f}%")

Na szczęście Snorkel oferuje dodatkowe narzędzia pozwalające na głębszą analizę wyniku zastosowania funkcji etykietujących.

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

Znaczenie poszczególnych kolumn jest następujące:
- `Polarity`: zbiór etykiet zwracanych przez funkcję
- `Coverage`: procent przykładów dla których funkcja zwraca wartość inną niż `ABSTAIN`
- `Overlaps`: procent przykładów dla których co najmniej jedna inna funkcja etyketująca zwróciła wartość
- `Conflicts`: procent przykładów dla których co najmniej jedna inna funkcja etyketująca zwróciła inną wartość

Gdyby zbiór uczący zawierał etykiety, to metoda zwróciłaby także:
- `Correct`: liczba poprawnych etykietowań
- `Incorrect`: liczba błędnych etykietowań
- `Empirical Accuracy`: procent poprawnych etykietowań

Sprawdźmy przykłady etykietowane przez funkcję `free()` jako spam

In [ ]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

Wydaje się, że dobrym wskaźnikiem dla spamu jest też fraza "call now". Dodajmy zatem jeszcze jedną funkcję etykietującą.

In [ ]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

Zobaczmy, które przykłady zostały etykietowane jako spam przez funkcję `call_now()` ale pominięte przez `free()`.

In [ ]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2])
df_train.iloc[buckets[(ABSTAIN, SPAM)]].sample(10, random_state=1)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

# Ćwiczenie 1 (1 pkt.)

Napisz funkcję etykietującą, która oznacza jako spam wszystkie wiadomości zawierające słowo "HOT" pisane kapitalikami.

In [ ]:
@labeling_function()
def hot(sms):
    ...

### Wyszukiwanie na podstawie wyrażenia regularnego

Kolejnym rodzajem funkcji etykietującej jest funkcja wykorzystująca regexp do znalezienia określonych wyrażeń.

In [ ]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"I\s.*free", sms.text, flags=re.I):
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, hot]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

Porównajmy przykłady, które funkcja `free()` etykietuje jako spam, a funkcja `regex_I_am_free()` uznaje za poprawne

In [ ]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

### Wyszukiwanie na podstawie heurystyki

Prostą heurystyką pozwalającą na znalezienie spamu jest przyjęcie, że jeśli ponad 10% tekstu sms-a jest napisana kapitalikami, to jest duża szansa, że jest to spam.

In [ ]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

# Ćwiczenie 2 (1 pkt.)

Zapisz funkcję etykietującą która oznaczy jako poprawne te wiadomości, które są krótsze niż 10 słów i nie zawierają żadnego słowa napisanego kapitalikami.

In [ ]:
@labeling_function()
def short_and_no_uppercase(sms):
  ...

### Wykorzystanie zewnętrznego modelu statystycznego

W trakcie etykietowania danych można posłużyć się zewnętrznymi modelami, których odpowiedź może być istotną informacją dla podjęcia decyzji o etykiecie przykładu. Snorkel posiada kilka wbudowanych integracji pod postacią interfejsu `Preprocessor`, na poniższym przykładzie wykorzystamy bibliotekę `SpaCy` do przeprowadzenia dodatkowej analizy gramatycznej tekstu. Konieczne będzie jednak ściągnięcie modelu języka angielskiego

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!python -m spacy validate

In [ ]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Przyjmijmy, że krótkie sms-y w których pojawia się odniesienie do konkretnej osoby nie są spamem.

In [ ]:
@labeling_function(pre=[spacy])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

Innym przykładem wstępnego przetworzenia danych na potrzeby etykietowania będzie wyznaczenie średniej częstości słów w dokumencie. Poniżej definiujemy funkcję wyznaczającą średnią częstość słów i dekorujemy ją jako przykład pre-procesora. W momencie wysłania sms-a do kolejnej funkcji etykietującej, pre-procesor uzupełni sms o średnią częstotliwość słów i na tej podstawie funkcja etykietująca podejmie decyzję (zakładamy, że jeśli sms zawiera wiele rzadkich słów to jest spamem).

In [ ]:
!pip install wordfreq

In [ ]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [ ]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

# ćwiczenie 4 (2 pkt.)

Napisz funkcję etykietującą, która oznaczy jako spam wiadomości zawierające więcej niż 3 przymiotniki. Wykorzystaj bibliotekę SpaCy do pre-processingu. 

_Podpowiedź_: poniższy przykład pokazuje, w jaki sposób można odczytać oznaczenie części mowy dla każdego tokenu z analizowanej wiadomości. Informacje o wszystkich właściwościach tokenów rozpoznawanych przez SpaCy można znaleźć w [dokumentacji API](https://spacy.io/api/token)

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

In [ ]:
# kod pomocniczy

POS_counts = nlp(sms).count_by(spacy.attrs.POS)
for k,v in sorted(POS_counts.items()):
    print(f'{k:{4}}. {nlp(sms).vocab[k].text:{5}}: {v}')

In [ ]:
@labeling_function()
def three_adjectives(sms):
    ...

## Połączenie funkcji etykietujących do postaci jednego modelu

Celem funkcji etykietujących nie jest uzyskanie indywidualnie dużego pokrycia. Funkcje etykietujące z natury rzeczy są zaszumione i mogą dokonywać wielu indywidualnych błędów. Prawdziwa użyteczność funkcji etykietujacych staje się oczywista w momencie w którym wiele funkcji zostanie połączonych do postaci jednego modelu.

W pierwszej kolejności zbudujemy prosty model oparty na głosowaniu większościowym, a następnie zbudujemy bardziej złożony model. 

In [ ]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [ ]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

In [ ]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

In [ ]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Dokładność głosowania większościowego:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Dokładność modelu probabilistycznego:':<25} {label_model_acc * 100:.1f}%")

Niestety, niektóre punkty danych nie otrzymają żadnej etykiety. Przed wysłaniem wyniku etykietowania do dalszego przetwarzania konieczne jest odfiltrowanie tych punktów.

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

Jak widać, udało się w szybki sposób przygotować etykiety dla około 720 przykładów (przypomnijmy, że początkowo żaden przykład w zbiorze `df_train` nie posiadał etykiet).

Następnym krokiem będzie wykorzystanie przygotowanych etykiet jako danych uczących dla faktycznego klasyfikatora. Posłużymy się prostą [regresją logistyczną](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), wcześniej dokonując przetworzenia danych wejściowych. Ponieważ pracujemy z tekstem, wykorzystamy [wektorową reprezentację słów](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) stworzoną na podstawie 5-gramów przez `TfidfVectorizer`.

In [ ]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import TfidfVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = TfidfVectorizer(ngram_range=(1,1))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [ ]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(max_iter=500)
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(max_iter=500)

In [ ]:
print(f"Dokładność regresji logistycznej: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

# Ćwiczenie 5 (0.5 pkt)

Czy ostateczny model poprawił wynik w stosunku do głosowania większościowego i modelu `LabelModel`? Znacznie / nieznacznie?

-- Twoja odpowiedź --

# Ćwiczenie 6 (1 pkt)

Dodaj napisane przez siebie funkcje etykietujące do listy lfs, po raz kolejny wytrenuj model zbiorczy nadający etykiety szkoleniowe, pobierz te etykiety i  wytrenuj regresję logistyczną dla wektora zliczeń słów. 
Ile tym razem próbek udało się oznaczyć? Czy polepszyło to klasyfikację? 

In [ ]:
lsf = ...

In [ ]:
# kod nadawania etykiet szkoleniowych, wektoryzacji i treningu regresji logistycznej, 
# wypisywania metryk dla modelu  ...

# Przykład word2vec

Na koniec - krótki eksperyment z word2vec. 
Biblioteka [SpaCy](https://spacy.io/) posiada wbudowane, przeliczone wektory dystrybucyjne słów o długości 300 (pomimo rozległego słownika, niektóre słowa nie będą miały wektora - do sprawdzenia, czy wektor dla słowa istnieje służy metoda is_oov *is out of vocabulary?*).

Wektor otrzymujemy przy użyciu metody `.vector`.

*Gdy wywołamy metodę `.vector` na całym dokumencie/fragmencie tekstu, otrzymamy uśredniony wektor. Nie zawsze będzie to dawało sensowny wynik - np. dystrybucja słowa 'food' i 'fast' mogą się znacznie różnić - co zatem będzie reprezentował uśredniony wektor dla 'fast food'? Statyczne wektory, takie jak te otrzymywane dzięki algorytmowi word2vec nie rozwiązują tego problemu, dlatego aktualnie najpopularniejsze są dystrybucje kontekstowe, takie jak rodzina algorytmów BERT.*

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
nlp(u'lion').vector # word embedding dla słowa

In [ ]:
nlp(u'lion is here').vector # uśredniony word embedding dla ciągu słów

Sprawdźmy, czy mimo prostoty modelu i statyczności dystrybucji word2vec jest w stanie pokazać nam rzeczywiste zależności semantyczne niektórych słów:

In [ ]:
tokens = nlp(u'lion cat truck bus love hate')

# sprawdźmy krzyżowo podobieństwo słów
for t1 in tokens:
  for t2 in tokens:
    if t1!=t2: # para takich samych tokenów zawsze zwróci 1
      print(t1.text, t2.text, t1.similarity(t2))

# ćwiczenie 7 (0.5 pkt)

Przyjrzyj się wynikom. Czy są sensowne?
Jak myślisz, dlaczego 'love' i 'hate' mają tak wysoki wynik podobieństwa?

-- Twója odpowiedź --


Plik ipynb prześlij na adres ***